<a href="https://colab.research.google.com/github/jfjofilipe/Aulas_2024/blob/main/salesman_pso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Otimização por Enxame de Partículas**
> A Otimização por Enxame de Partículas, *Particle Swarm Optimization* - PSO, simula o comportamento social de um bando de pássaros à procura de um alvo (alimento, local para pouso, proteção contra predadores, entre outros) (Figura 1).

Figura 1. Inspiração natural do PSO

<img src="https://drive.google.com/uc?id=1Djc8LR0wVPiqW3LJfCKI5-UNPTrlUDuT" width="500">

> A PSO é um conjunto de algoritmos populacionais
* A população é chamada de “enxame”
* Os indivíduos são chamados de “partículas”

> As Partículas possuem posição e velocidade (A posição
muda ao longo do tempo). As partículas “voam” pelo espaço de busca multidimensional. Durante o “voo”, cada
partícula ajusta sua posição de acordo com sua experiência e a de partículas vizinhas. Apesar da estrutura simples dos indivíduos, o exame, graças a cooperação entre esses, é capaz de resolver problemas complexos.

> O presente notebook colab, escrito por Alison Zille Lopes, pretende introduzir à PSO. A codificação foi baseada na biblioteca PySwarms e na documentação presente em {1}.
>* Abaixo podemos visualizar a importação dos pacotes e módulos necessários.
---
{1} MIRANDA, L., J., V. Welcome to PySwarms's documentation. Disponível em: https://pyswarms.readthedocs.io/en/latest/index.html. Acesso em: 23 abr 2021.

In [ ]:
#[1] Importando pacotes e módulos

# instalando pacote
!pip install pyswarms

# importando módulos
import sys
import numpy as np
import matplotlib.pyplot as plt
import random

# importando PySwarms
import pyswarms as ps

     |████████████████████████████████| 104 kB 37.2 MB/s 


#**Aplicação:** Solução do Problema do Caixeiro Viajante
> O algoritmo original da otimização por enxame de partículas foi criado para a solução de problemas que envolvam variáveis contínuas. Entretanto, como muitos problemas do mundo real são discretos, existem versões modificadas do PSO, como o BinPSO, para a aplicação a esses problemas.

> Assim, o clássico **Problema do Caixeiro Viajante** foi escolhido para a aplicação do BinPSO. O **Problema do Caixeiro Viajante**, ou *Travelling Salesman Problem*, reside no objetivo de encontrar a menor rota possível para visitar um conjunto de cidades, passando por cada uma delas uma única vez, e retornar à origem.
* O espaço de estados para esse problema pode ser representado por um grafo completamente conexo. Os vértices são as cidades e as arestas representam vias entre cidades, havendo uma distância (custo) associada.
* O trecho de código abaixo gera um grafo para o problema do caixeiro viajante.
  * O usuário pode escolher o número de cidades.
  * O grafo é gerado como uma lista de arestas, sendo as arestas tuplas compostas pelos vértices e o comprimento da aresta.
    * (v1, v2, dist), sendo v1 e v2 vértices e dist o comprimento da aresta.
    * O comprimento da aresta é definido aleatoriamento no intervalo [10, 100].

> A função de aptidão, ou função custo, também está presente na codificação abaixo. Essa, chamada evalRoutes, recebe as posições de cada partícula (rotas), retornando o custo.
* As posições são sequências binárias, baseadas na representação do grafo anterior, que indicam quais arestas fazem parte da rota.

In [ ]:
#[2] Geração do Grafo e Função de Custo

# função graphTSP(numCities, minDist, maxDist)
# parâmetros:
#   numCities: número de cities
#   minDist: menor valor de distância
#   maxDist: maior valor de distância
# retorno:
#   roads: lista de estradas - arestas: (cidade1, cidade2, distância). As distâncias
#   entre duas cidades são determinadas aleatoriamente entre minDist e maxDist
def graphTSP(numCities, minDist, maxDist):
  roads = []
  for i in range(numCities):
    for j in range(i+1, numCities):
      roads.append((i, j, random.randint(minDist, maxDist)))
  return roads

numCities = 5     #  Número de cidade inicial

while(True):
  numCities = int(input('Digite o número de cidades [>4]: '))
  if (numCities > 4):
    break
  else:
    print('O número de cidades deve ser maior que 4!')

roads = graphTSP(numCities, 10, 100)


# Função evalRoutes(particles)
# parâmetros:
#   particles: array (numpy.ndarray) de posições das partículas (rotas)
# retorno:
#   costs: lista de custos para cada partícula.
def evalRoutes(particles):
  costs = []
  for individual in particles:
    cost = 0;

    validArcs = np.where(individual == 1)[0]

    for arc in validArcs:
      cost = cost + roads[arc][2]

    for i in range(numCities):
      count = 0
      for arc in validArcs:
        if (roads[arc][0]==i or roads[arc][1]==i):
          count = count + 1
      if(count!=2):
        cost = cost + 10000

    if(sum(individual)!=numCities):
      cost = cost + 10000
    else:
      # identificar a ocorrência de ciclos
      validArcs = list(validArcs)
      current = validArcs[0]
      visited = [roads[current][0]]
      while(len(validArcs)>1):
        node = roads[current][1]
        if (not roads[current][0] in visited):
          node = roads[current][0]
        elif (roads[current][1] in visited):
          cost = cost + 10000
          break
        visited.append(node)
        validArcs.remove(current)
        for arc in validArcs:
          if (node == roads[arc][0] or node == roads[arc][1]):
            current = arc
            break

    costs.append(cost)

  return costs

print('Grafo:')
for road in roads:
  print(road)

Digite o número de cidades [>4]: 5
Grafo:
(0, 1, 66)
(0, 2, 31)
(0, 3, 75)
(0, 4, 44)
(1, 2, 35)
(1, 3, 28)
(1, 4, 77)
(2, 3, 96)
(2, 4, 30)
(3, 4, 33)


#**Processamento da PSO**
>O processamento do algoritmo básico da otimização por enxame de partículas é expresso de forma simplificada na Figura 3.

Figura 3. PSO simplificado

<img src="https://drive.google.com/uc?id=1q3BNyAoAtXFem4y5Fe76A2qzB9A2_1t0" width="500">

>A Figura 4 traz o pseudocódigo para esse algoritmo, o qual baseia-se na notação:
* $x_i$, posição da partícula $i$
  * $x_i = [x_{i,1}; x_{i,2}; ...;x_{i,n}]$
* $v_i$, velocidade da partícula $i$
  * $v_i = [v_{i,1}; v_{i,2}; ...;v_{i,n}]$
* $f(x_i)$, aptição da partícula $i$
* $m$, tamanho da população (enxame) de partículas
* $p_i$, ($pbest_i$, $personal best$) a melhor posição encontrada pela partícula $i$
  * As partículas precisam “guardar” qual sua melhor posição.
* $g$, ($gbest$, $global best$) a melhor posição guardada por todas as partículas.
* $c_1$ e $c_2$, parâmetros cognitivos e sociais (também chamados de taxas de aprendizagem)
* $W$, ponderação de inércia
* $r_{1,d}$ e $r_{2,d}$, números aleatórios entre 0 e 1

Figura 4. Pseudocódigo da PSO

<img src="https://drive.google.com/uc?id=14lR-elAxd2CedG--aYjhREG2IumxreOk" width="700">

>A atualização da velocidade pode ser expressa por Eq. 1:
* $v_{i,d}(t+1)=W.v_{i,d}(t)+c_1r_{1,d}(p_{i,d}(t)-x_{i,d}(t))+c_2r_{2,d}(g_d(t)–x_{i,d}(t))$
  * O impacto de $W$ depende, também, dos valores selecionados para $c_1$ e $c_2$. A escolha desses parâmetros determinará se o enxame irá se concentrar na livre exploração ou em aproveitar regiões com boas soluções já encontradas.

> Uma vez atualizada a velocidade da partícula $i$, é hora de atualizar a posição por Eq. 2 (Equação da posição para BinPSO):
* $x_{i,d}(t + 1) = \{1, se\ U(0,1)<sig(v_{i,d}(t+1)); 0,\ em\ caso\ contrário\}$
  * $U(0,1)$ é um valor aleatório entre 0 e 1
  * $sig(v_{i,d}(t+1)) = {1 \over 1+exp(-v_{i,d}(t+1))}$

> A melhor localização da partícula ($p_i$) também é atualizada caso necessário:
* $p_i(t + 1) = p_i(t), se f(x_i(t)) ≤ f(p_i(t))$
* $p_i(t + 1) = x_i(t), se f(x_i(t)) > f(p_i(t))$

> Após a atualização das melhores posições de cada partícula, é verificado se a melhor posição global (g) precisa ser atualizada:
* Escolhe:
  * $p̂(t + 1)$ ∈ {$p_1(t + 1), ... ,p_n(t + 1)$}
* tal que:
  * $f(p̂(t + 1))$ = max{$f(p_1(t + 1)), ... ,f(p_n(t + 1))$}
* Assim:
  * $g(t + 1) = g(t), se f(g(t)) > f(p̂(t + 1))$
  * $g(t + 1) = p̂(t + 1), se f(g(t)) ≤ f(p̂(t + 1))$

> Através do uso do pacote PySwarms, os detalhes do funcionamento do algoritmo são transparentes ao usuário. Entretanto, ainda é preciso definir os hiperparâmetros utilizados pela equação de velocidade, bem como o número de partículas e iterações do algoritmo.

In [ ]:
#[3] Hiperparâmetros e Otimização

numParticles = 10

while(True):
  numParticles = int(input('Digite o número de partículas [>=10]: '))
  if (numParticles >= 10):
    break
  else:
    print('O número de partículas deve ser maior ou igual a 10!')

numIters = 10

while(True):
  numIters = int(input('Digite o número de iterações do algoritmo [>=10]: '))
  if (numIters >= 10):
    break
  else:
    print('Use no mínimo 10 iterações!')

# hiperparâmetros
options = {'c1': 0.5, 'c2': 0.3, 'w':1, 'k':(numParticles-1), 'p':1}

# instanciação de objeto da classe BinaryPSO
optimizer = ps.discrete.binary.BinaryPSO(n_particles=numParticles, dimensions=len(roads), options=options, velocity_clamp=(-3,3))

# execução da otimização
cost, pos = optimizer.optimize(evalRoutes, iters=numIters)

print('Custo = ', cost)

validArcs = np.where(pos==1)[0]
print('Arcos: ')
for arc in validArcs:
  print(roads[arc])

Digite o número de partículas [>=10]: 10
Digite o número de iterações do algoritmo [>=10]: 10


2021-06-08 19:56:24,178 - pyswarms.discrete.binary - INFO - Optimize for 10 iters with {'c1': 0.5, 'c2': 0.3, 'w': 1, 'k': 9, 'p': 1}
pyswarms.discrete.binary: 100%|██████████|10/10, best_cost=171
2021-06-08 19:56:24,222 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 171.0, best pos: [0 1 0 1 1 1 0 0 0 1]


Custo =  171.0
Arcos: 
(0, 2, 31)
(0, 4, 44)
(1, 2, 35)
(1, 3, 28)
(3, 4, 33)
